 **Wireshark Flow Analysis Project: Optimized Tracking of DNS, TCP, TLS Handshake, and HTTPS**

**🔹 Project Introduction**

This project is a continuation of our previous Wireshark flow analysis, where we developed an optimized methodology to track the resolution and connection process of a website.

Instead of following traditional chronological network analysis, which can be inefficient, we will apply a targeted workflow to efficiently confirm DNS resolution, TCP handshake, TLS handshake, and HTTPS communication for a new website.

The objective is to apply our refined tracking strategy and validate its effectiveness in real-world network analysis.

**🔹 Project Goals**
1️⃣ Efficiently identify how a website is accessed over the network.
2️⃣ Determine the exact DNS resolution used (IPv4 or IPv6, CNAME chains).
3️⃣ Verify if the connection is established over TCP (TLS) or QUIC (HTTP/3).
4️⃣ Track the TCP handshake to confirm the three-way handshake.
5️⃣ Explicitly follow the TLS handshake to analyze encryption details and certificate exchange.
6️⃣ Confirm encrypted HTTPS communication and ensure data security.
7️⃣ Optimize Wireshark's built-in filtering and search capabilities to eliminate unnecessary steps.

**🔹 Step-by-Step Process (Optimized Workflow)**
Using our established efficient tracking protocol, we will proceed as follows:

**1️⃣ Start With the First Connection Attempt (Find the IP)**
📌 Filter to Identify the First Network Connection:
tcp.flags.syn == 1 or udp.port == 443
This filter identifies the first connection attempt and tells us whether the site used TLS over TCP (HTTPS) or QUIC (HTTP/3). By checking the Destination IP in the first connection, we can determine whether the connection was established using an A (IPv4) or AAAA (IPv6) record.


**2️⃣ Work Backward to Find the DNS Resolution**
📌 Once the IP is identified, apply one of these filters to find the exact DNS response:
dns.aaaa == [IPv6 Address]  # If the site resolved using IPv6
dns.a == [IPv4 Address]      # If the site resolved using IPv4
✅ This eliminates manual tracking of multiple DNS queries and directly shows the response that resolved to the IP.
✅ If the DNS response contains CNAME records, we will analyze the CNAME resolution path.

**3️⃣ Track the TCP Handshake**
📌 Filter for TCP Handshake Packets (if using TCP):
ip.addr == [Resolved IP] and tcp.flags.syn == 1
✅ Confirms the three-way handshake (SYN → SYN-ACK → ACK).
✅ Ensures the TCP session was properly established.

**4️⃣ Explicitly Track the TLS Handshake**
📌 Filter for TLS Handshake Packets:
ip.addr == [Resolved IP] and tls.handshake
✅ Tracks the TLS version used (TLS 1.2 vs. TLS 1.3).
✅ Captures the Server Certificate exchange (verifying if the site uses a trusted CA).
✅ Confirms whether SNI (Server Name Indication) was used.
✅ Verifies key exchange and encryption setup.

**5️⃣ Confirm HTTPS Communication**
📌 Filter for the Encrypted HTTPS Data Flow:
ip.addr == [Resolved IP] and tls
✅ Confirms that an encrypted TLS session was established.
✅ Ensures that no plaintext HTTP traffic was used.
✅ Verifies that the secure session is actively transmitting data.

**🔹 Expected Outcome**
By following this optimized method, we will be able to: ✅ Efficiently analyze how a site is accessed without excessive manual searching.
✅ Confirm whether IPv4 or IPv6 was used in the connection.
✅ Determine if the site was contacted via TCP (TLS) or QUIC.
✅ Trace the entire sequence from DNS resolution to HTTPS communication.
✅ Explicitly confirm encryption details through the TLS handshake.

This project will serve as a validation of our refined methodology and demonstrate its repeatability in real-world network traffic analysis. 🚀

🔹 Why This Approach Is Superior to Traditional Analysis
🔹 Eliminates unnecessary manual tracking of multiple DNS queries.
🔹 Directly finds the actual IP used in the connection instead of guessing.
🔹 Works backward efficiently to find the exact DNS response.
🔹 Moves forward through all network layers (TCP, TLS, HTTPS) without confusion.
🔹 Drastically reduces time spent searching for packets in large captures.

This refined methodology should now be the standard for all future Wireshark-based network traffic analysis.

**Wireshark Flow Analysis - Step 1: Identifying Reebok’s Connection Type**

📌 Overview
In this step, we set out to determine how Reebok’s website was accessed—specifically, whether it used IPv4 or IPv6, and whether it communicated using QUIC (HTTP/3) or TLS over TCP (HTTPS).

🛠 Filter Applied
frame contains "reebok.com"

📌 Why We Used This Filter:

Initially, our attempts to filter by IPv6 (ip6.addr == [Reebok IPv6]) failed, as Wireshark did not recognize the IPv6 address.

Instead of continuing to troubleshoot IPv6 filtering, we pivoted to a more efficient approach by filtering for any frame containing "reebok.com."

This allowed us to quickly isolate a small set of relevant packets that explicitly referenced Reebok’s domain.

✅ Findings from This Filter
After applying the filter, we obtained 5 results, each representing communication between our system and Reebok.

1️⃣ We identified a key packet (Frame 65) that contained the following details:

Source: 192.168.1.185 (Local Computer)

Destination: 23.227.38.65 (Reebok’s Server)

Protocol: TCP (Not QUIC)

Port: 443 (HTTPS)

TCP Flags: ACK

2️⃣ Conclusions from Frame 65:

Reebok used IPv4 (A record) for this connection – the destination was an IPv4 address.

Reebok is NOT using QUIC (HTTP/3), but rather traditional TLS over TCP (HTTPS).

The presence of an ACK packet suggests that we need to analyze the full TCP handshake next.

🚀 Next Steps
Now that we have confirmed that Reebok is using TLS over TCP, our next step is to analyze the full three-way handshake.

📌 Next Filter to Apply:
(ip.addr == 23.227.38.65) and (tcp.flags.syn == 1 or tcp.flags.ack == 1)
✅ This will show:

The SYN packet (client’s request to initiate a connection).

The SYN-ACK response from the server.

The ACK packet completing the handshake.

Once we confirm the TCP handshake, we will proceed to the TLS handshake to analyze encryption setup.

**🔹 Summary**
Filter Used: frame contains "reebok.com"

Findings: Reebok uses IPv4 (A record) and TLS over TCP (HTTPS), NOT QUIC.

Next Step: Analyze the TCP handshake using tcp.flags.syn == 1 or tcp.flags.ack == 1.

**Wireshark Flow Analysis - Step 2: Confirming TCP Handshake Completion**

**Objective**

To establish a secure connection, the TCP three-way handshake must be completed before encryption can begin. This step verifies the successful initiation of the connection between the local machine and Reebok’s server.

Filter Applied
(ip.addr == 23.227.38.65) and (tcp.flags.syn == 1 or tcp.flags.ack == 1)
This filter isolates packets related to the TCP handshake process, specifically SYN, SYN-ACK, and ACK packets. The goal is to confirm that the handshake was completed successfully, allowing the transition to the encryption phase.

Findings
Application of the filter resulted in 21 frames, with the key handshake sequence identified as follows:

1️⃣ Frame 47 (SYN) → Connection request sent from the local machine to Reebok’s server (23.227.38.65).
2️⃣ Frame 63 (SYN-ACK) → Reebok’s server acknowledges the connection request.
3️⃣ Frame 64 (ACK) → The local machine completes the handshake by acknowledging the SYN-ACK.

Frame 64 was identified as the final step in the handshake, confirming the connection was established.

Conclusion
The TCP connection was successfully established between the local machine and Reebok’s server.

Communication is occurring over IPv4 (23.227.38.65) on TCP port 443 (HTTPS).

No handshake errors or retransmissions were detected.

Next Steps: Analyzing the TLS Handshake
With the TCP connection confirmed, the next phase involves analyzing the TLS handshake, where encryption settings are negotiated.

Filter to Apply
(ip.addr == 23.227.38.65) and tls.handshake
This filter isolates TLS handshake packets, specifically:

Client Hello → The local machine proposes encryption methods.

Server Hello → Reebok’s server selects encryption settings.

Certificate Exchange → Reebok’s server provides its SSL/TLS certificate.**

**Wireshark Flow Analysis - Step 3: Analyzing the TLS Handshake**

**Objective**

Following the successful establishment of the TCP connection, the next step is to verify the TLS handshake and confirm encryption details. The Server Hello message provides key information about the negotiated security settings, including the TLS version, selected cipher suite, and key exchange method.

Filter Applied
No additional filter was applied.
The TLS handshake packets, including Client Hello and Server Hello, were already visible from the previous filtering step. The Server Hello was identified in Frame 73 and analyzed directly.

Findings
Frame 73 was identified as the Server Hello message, confirming the encryption details for the session.

1️⃣ TLS Version: TLS 1.3 (0x0304)

TLS 1.3 removes older cryptographic vulnerabilities and optimizes security by streamlining the handshake process.

Backward compatibility with TLS 1.2 is maintained through a Change Cipher Spec message, but it is not actively used in TLS 1.3 negotiations.

2️⃣ Cipher Suite Selected:
TLS_AES_128_GCM_SHA256 (0x1301)

AES-128 in GCM (Galois/Counter Mode) provides authenticated encryption (AEAD).

SHA-256 ensures message integrity.

This cipher suite supports Perfect Forward Secrecy (PFS), ensuring that past communications remain secure even if a session key is compromised.

3️⃣ Key Exchange Mechanism:

Key Share Extension: Unknown (4588)

This likely corresponds to X25519 or P-256 (ECDHE), confirming the use of ephemeral key exchange for Perfect Forward Secrecy (PFS).

4️⃣ Session ID and Random Value:

A 32-byte Session ID and random value were exchanged, further securing session negotiation.

Conclusion
TLS 1.3 is in use, providing modern encryption and eliminating legacy weaknesses.

The cipher suite TLS_AES_128_GCM_SHA256 was selected, ensuring authenticated encryption and Perfect Forward Secrecy.

The key exchange process indicates the use of an ephemeral algorithm, reinforcing security.

Next Steps: Analyzing the Key Exchange Process

With the TLS version and cipher suite confirmed, the next step is to analyze the key exchange details, specifically how session keys are exchanged.

Filter to Apply

(ip.addr == 23.227.38.65) and tls.handshake.type == 16

This filter isolates the Client Key Exchange message, which contains:

The specific ephemeral key exchange algorithm used (likely X25519 or P-256).

The session key details used for symmetric encryption.

Verification of Perfect Forward Secrecy (PFS).

**Wireshark Flow Analysis - Step 4: Completion of the TLS Handshake & Encryption Activation**

**Objective**

Following the verification of the Server Hello and cipher suite selection, the next step was to confirm completion of the TLS handshake and activation of encrypted communication. This was achieved by identifying the Change Cipher Spec and Application Data messages, signaling that encryption was now active.

Filter Applied
No additional filter was applied.

The TLS handshake packets remained visible from the previous filtering steps used in TCP and TLS handshake analysis.

The Change Cipher Spec and Application Data messages were located directly within the dataset.

Findings

Frame 77 was identified as a key packet marking the transition to encrypted HTTPS communication.

1️⃣ TLS Handshake Completed

The presence of "Change Cipher Spec" indicates that encryption keys have been exchanged.

TLSv1.3 Record Layer: Application Data confirms that further communication is now encrypted.

2️⃣ Cipher Suite & Encryption Activation

The previously identified TLS_AES_128_GCM_SHA256 cipher suite is now in effect.

Authenticated encryption (AEAD) is active.

3️⃣ Perfect Forward Secrecy (PFS) Is Confirmed

Since AES-GCM with ephemeral key exchange automatically ensures PFS, there was no need to search for session key details.

The Key Share extension in Frame 73 indicated an ephemeral key exchange method, further reinforcing PFS.

4️⃣ Key Exchange Method

The Server Hello (Frame 73) included a Key Share extension with an identifier (Unknown (4588)).

This most likely corresponds to X25519 or P-256 (ECDHE), both of which support PFS.

Since TLS 1.3 does not explicitly expose session key details, there was no need to apply an additional filter to confirm key exchange.

**Conclusion**

Encryption is now fully active, and all further communication is encrypted.

TLS 1.3 has successfully negotiated a secure session with PFS.

No additional filtering was required to confirm session key details, as TLS 1.3 does not expose them.

**📌 Should We Analyze HTTPS Further?**

✅ No, further HTTPS analysis would not yield additional readable data.

Since TLS 1.3 encrypts the payload, any further Application Data packets cannot be analyzed without the private key.

We have already confirmed all critical security aspects (TLS version, cipher suite, key exchange, encryption activation).

✅ No additional filters are required for HTTPS.

The only way to extract more data would be decrypting HTTPS traffic, which requires private key access (not possible in this scenario).

**🚀 Conclusion: We have all the HTTPS information needed. No further analysis is required.**

**Next Steps: Analyzing the DNS Resolution Process**

Now that encryption is confirmed, the next step is to pivot back to DNS resolution to determine:

How Reebok’s IP address (23.227.38.65) was obtained.

What DNS records were retrieved.

Filter to Apply

dns and dns.flags.response == 1

✅ This will display all DNS responses, allowing verification of the resolved IP address.

**Wireshark Flow Analysis - Step 5: DNS Resolution Process**


Objective
The final step in the analysis is to verify how Reebok’s domain name (reebok.com) was resolved to an IP address. This confirms the Domain Name System (DNS) response that allowed the connection to be established.

Filter Applied

dns and dns.flags.response == 1 and dns.qry.name contains "reebok.com" and dns.qry.type == 1

This filter ensures that only actual DNS responses related to reebok.com are displayed, removing unrelated queries and IPv6 results.

**Findings**

📌 The DNS resolution process was identified in Frame 58, confirming the translation of the domain name reebok.com into an IPv4 address.

1️⃣ DNS Query Details

Transaction ID: 0xc751

DNS Query Type: A (IPv4 Address)

Recursion was requested and provided (Recursion desired: 1, Recursion available: 1).

The authoritative bit was not set, meaning the response came from a caching or recursive resolver rather than Reebok’s authoritative name server.

2️⃣ DNS Answer Record (A Record)

Domain Queried: reebok.com

Resolved IP Address: 23.227.38.65

Class: IN (Internet)

3️⃣ DNS Server Details

Source Address: dsldevice6.attlocal.net (2600:1700:a3a0:d90::1)

This indicates that the local router (ATT DSL device) handled the DNS resolution request before returning the result to the client.

4️⃣ Time to Resolution

0.079555000 seconds

This is a fast DNS resolution time, meaning the resolver likely had this record cached rather than querying an upstream authoritative server.

Conclusion
✅ The DNS resolution process successfully translated reebok.com into 23.227.38.65, which was then used for the HTTPS connection.
✅ The query was handled by the local router’s DNS resolver (dsldevice6.attlocal.net).
✅ Recursion was performed, meaning the resolver either retrieved or cached the record from an upstream server.





**📌 Wireshark Flow Analysis: Full Reference Summary**

Efficient Tracking of DNS, TCP, TLS, and HTTPS Using Targeted Filtering

**Objective:** This structured methodology provides an efficient and repeatable approach to analyzing secure web traffic, minimizing unnecessary data and refining filtering to obtain only pertinent results. This ensures a clear, systematic method that avoids defaulting to general or inefficient filtering.

**🔹 Step 1: Identifying the Initial Connection Attempt**

Purpose:

To determine whether the connection was made via IPv4 (A record) or IPv6 (AAAA record), and whether it used QUIC (UDP) or traditional TLS over TCP.

Filter Applied:
tcp.flags.syn == 1 or udp.port == 443
Why?
Filters only SYN packets (starting a TCP handshake) and UDP packets on port 443 (possible QUIC).

Immediately reveals whether the connection is TCP/TLS or UDP/QUIC.

Also indicates whether IPv4 (A record) or IPv6 (AAAA record) was used.

Findings:
✅ The website used IPv4 (A record) over TCP, confirming TLS-based HTTPS rather than QUIC.
✅ The first connection attempt was to 23.227.38.65 (Reebok’s IPv4 address).

**🔹 Step 2: Analyzing the TCP Handshake**
Purpose:
To confirm that the 3-way TCP handshake (SYN, SYN-ACK, ACK) was successful, ensuring a stable connection was established.

Filter Applied:

(ip.addr == 23.227.38.65) and (tcp.flags.syn == 1 or tcp.flags.ack == 1)
Why?
Filters out irrelevant traffic, keeping only TCP handshake packets related to Reebok’s IP.

Ensures the entire 3-way handshake process can be observed.

Findings:
✅ SYN, SYN-ACK, and ACK were present, confirming a complete handshake.
✅ Connection was established on port 443 (HTTPS).

**🔹 Step 3: Verifying the TLS Handshake & Encryption**

Purpose:
To confirm TLS 1.3 is being used, identify the cipher suite, and verify encryption activation.

Filter Applied:
(ip.addr == 23.227.38.65)

(No need for a more restrictive filter as the TLS handshake packets were already visible.)

Why?
The TLS handshake packets were already within our previously filtered dataset.

We located Client Hello (Frame 64) and Server Hello (Frame 73) manually without extra filtering.

Findings:
✅ TLS 1.3 (0x0304) was confirmed as the encryption protocol.
✅ Cipher Suite Selected: TLS_AES_128_GCM_SHA256 (Provides authenticated encryption and Perfect Forward Secrecy).
✅ Key Exchange: Server Hello contained a Key Share extension, confirming ephemeral key exchange (likely X25519).

**🔹 Step 4: Confirming the Transition to HTTPS Encryption**
Purpose:
To verify that encryption was fully activated and that the connection transitioned to encrypted HTTPS traffic.

Filter Applied:
(No additional filter was needed—this was visible from previous filtering.)

Key Packet:
✅ Frame 77 contained:

Change Cipher Spec message (indicating encryption activation).

Application Data (meaning all further communication is now encrypted).

Findings:
✅ TLS 1.3 handshake was complete; encryption is active.
✅ All further data is encrypted HTTPS traffic and cannot be analyzed without decryption keys.

**🔹 Step 5: Verifying DNS Resolution**

Purpose:
To confirm how Reebok’s domain (reebok.com) was resolved to an IP address (23.227.38.65).

Filter Applied:
dns and dns.flags.response == 1 and dns.qry.name contains "reebok.com" and dns.qry.type == 1
Why?
Filters only actual DNS responses (not queries).

Filters only DNS responses related to reebok.com.

Filters only IPv4 (A record) responses, excluding unnecessary IPv6 (AAAA) records.


**📌 Understanding How the DNS Response Filter Works**

This section explains two key filtering behaviors in Wireshark that are essential for accurately analyzing DNS resolution. These clarifications ensure that filtering remains precise and efficient, avoiding unnecessary packet noise.

**🔹 Question 1: Why Are Queries Not Included If dns.qry.name Is in the Filter?**

📌 Key Takeaways:

The filter we used includes dns.flags.response == 1, which ensures that only DNS responses are displayed and that queries are automatically excluded.

Even though dns.qry.name exists in both DNS queries and responses, this does not override dns.flags.response == 1.

Result: This filter only displays DNS responses that contain reebok.com, ensuring that all unrelated DNS queries are removed.

📌 Rule to Remember:
✅ If dns.flags.response == 1 is in the filter, DNS queries (dns.flags.response == 0) are always excluded.

**🔹 Question 2: How Does This Filter Exclude IPv6 (AAAA) Records?**
📌 Key Takeaways:

DNS query types are assigned specific numbers:

IPv4 (A) = 1

IPv6 (AAAA) = 28

The filter contains dns.qry.type == 1, which means only IPv4 (A) records are shown and all IPv6 (AAAA) records are excluded.

Even though dns.qry.type == 1 appears in both queries and responses, queries are already removed by dns.flags.response == 1.

📌 Rule to Remember:
✅ If dns.qry.type == 1 is in the filter, only IPv4 (A) records will appear, and IPv6 (AAAA) records will be automatically excluded.

🚀 Final Summary: The Complete DNS Filter Logic

dns.flags.response == 1 and dns.qry.name contains "reebok.com" and dns.qry.type == 1
✅ dns.flags.response == 1 → Ensures that only responses (not queries) are displayed.
✅ dns.qry.name contains "reebok.com" → Ensures that only responses related to reebok.com appear.
✅ dns.qry.type == 1 → Ensures that only IPv4 (A) records appear, excluding IPv6 (AAAA).

🚀 By combining these three conditions, this filter precisely captures the DNS resolution response for reebok.com, removing all unnecessary traffic.





**Findings:**
✅ Reebok.com was resolved to 23.227.38.65.
✅ The DNS request was handled by a local resolver (dsldevice6.attlocal.net).
✅ Resolution time: 0.079555000 seconds (likely cached).

**🔹 Conclusion: The Full Network Flow (DNS ➝ TCP ➝ TLS ➝ HTTPS)**

✅ DNS Query: reebok.com was resolved to 23.227.38.65
✅ Connection Type: IPv4, TCP-based (not QUIC)
✅ TCP Handshake: SYN, SYN-ACK, ACK confirmed
✅ TLS Handshake: TLS 1.3, TLS_AES_128_GCM_SHA256 cipher suite, Perfect Forward Secrecy
✅ Encryption Activation: TLS handshake completed, Change Cipher Spec and encrypted HTTPS traffic confirmed

**🔹 Strategy Overview: Why This Approach Works**

📌 Why Not Start With DNS?

We first verified that an actual connection was established before checking DNS.

If we had started with DNS, we would have captured all DNS responses (not just the relevant one).

By filtering only after confirming the IP used, we reduced noise and unnecessary steps.

📌 Why Use These Filters Instead of Broad Queries?

Filtering at each layer (DNS, TCP, TLS) ensures minimal irrelevant traffic.

More precise filtering leads to more efficient analysis and eliminates manual searching.

📌 How This Improves Over Traditional Network Analysis

Traditional Wireshark analysis often follows a broad, chronological approach.

This structured method narrows the scope at each step, ensuring only relevant packets are analyzed.


**📌 Summary: A Repeatable, Optimized Approach**
✅ Use filtering to analyze secure network traffic efficiently.

✅ Avoid broad captures; target specific records to eliminate noise.

✅ Follow the network flow in a logical sequence to confirm each stage.

🚀 This structured approach provides a clear, repeatable process for analyzing web traffic efficiently in Wireshark.